In [1]:
from copy import deepcopy
from itertools import combinations
import pickle
import typing as tp

from lightfm import LightFM
from lightfm.data import Dataset as LFMDataset
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.preprocessing import normalize
from transliterate import translit

In [2]:
FOLDER_DATA = "./data/"
RANDOM_STATE = 42

## Загрузка данных

In [3]:
interactions = pd.read_csv(FOLDER_DATA + "interactions_kion.csv")
items = pd.read_csv(FOLDER_DATA + "items_kion.csv")

Определим последний доступный день как тестовый, а 3 месяца до него - обучающей выборкой. Для простоты удачным взаимодействием будем считать просмотр любой длительности и при обучении будем использовать только эту информацию, игнорируя данные о признаках пользователей и контента.

In [4]:
train = interactions.loc[
    (interactions["last_watch_dt"] >= "2021-06-05")
    & (interactions["last_watch_dt"] <= "2021-09-05"),
    ["user_id", "item_id"]
]

test = interactions.loc[interactions["last_watch_dt"] == "2021-09-06", ["user_id", "item_id"]]
test = test.loc[test["user_id"].isin(train["user_id"])]

In [5]:
train = pd.merge(train, items[["item_id", "title", "genres"]], on="item_id")
test = pd.merge(test, items[["item_id", "title", "genres"]], on="item_id")

## Добавление аватаров в обучающую выборку

Далее для построения рекомендаций будем использовать алгоритм матричного разложения из библиотеки LightFM. Для формирования рекомендаций для аватаров необходимо, чтобы действия аватаров были в обучающей выборке.

Отметим, что добавляя действия аватаров в обучающую выборку, мы несколько изменяем ее распределение. Обычно этим можно пренебречь, т.к. объем обучающей выборки (в нашем случае 5 млн наблюдений) значительно больше объема действий аватаров (10 наблюдений).

In [6]:
titles = [
    "Джон Уик", 
    "Заложница", 
    "Перевозчик",
    "Форсаж: Хоббс и Шоу",
    "Терминатор 3: Восстание машин"
]
avatar_interactions_action = pd.DataFrame({"user_id": "avatar_action", "title": titles})
avatar_interactions_action = avatar_interactions_action.merge(items[["item_id", "title", "genres"]], on="title")
avatar_interactions_action

,user_id,title,item_id,genres
0,avatar_action,Джон Уик,7671,"боевики, триллеры"
1,avatar_action,Заложница,14362,"боевики, триллеры"
2,avatar_action,Перевозчик,8350,"боевики, триллеры, криминал"
3,avatar_action,Форсаж: Хоббс и Шоу,2323,"боевики, триллеры"
4,avatar_action,Терминатор 3: Восстание машин,2925,"боевики, фантастика, триллеры"


In [7]:
titles = [
    "Тупой и еще тупее 2",
    "Типа крутые легавые",
    "Голый пистолет",
    "Убойные каникулы",
    "Карты, деньги, два ствола"
]
avatar_interactions_comedy = pd.DataFrame({"user_id": "avatar_comedy", "title": titles})
avatar_interactions_comedy = avatar_interactions_comedy.merge(items[["item_id", "title", "genres"]], on="title")
avatar_interactions_comedy

,user_id,title,item_id,genres
0,avatar_comedy,Тупой и еще тупее 2,8391,комедии
1,avatar_comedy,Типа крутые легавые,2915,"боевики, триллеры, детективы, комедии"
2,avatar_comedy,Голый пистолет,1734,комедии
3,avatar_comedy,Убойные каникулы,4979,"ужасы, комедии"
4,avatar_comedy,"Карты, деньги, два ствола",2866,комедии


In [8]:
train = pd.concat([train, avatar_interactions_action, avatar_interactions_comedy], sort=False)

## Обучение модели

In [15]:
# lfm_dataset = LFMDataset()
# lfm_dataset.fit(
#     users=train["user_id"].values,
#     items=train["item_id"].values,
# )
# with open("lfm_dataset.pkl", 'wb') as f:
#     pickle.dump(lfm_dataset, f)
#
with open("lfm_dataset.pkl", 'rb') as f:
    lfm_dataset = pickle.load(f)

train_matrix, _ = lfm_dataset.build_interactions(zip(*train[["user_id", "item_id"]].values.T))

In [11]:
# lfm_model = LightFM(
#     learning_rate=0.01, 
#     loss='warp', 
#     no_components=64,
#     random_state=RANDOM_STATE
# )
# lfm_model.fit(
#     interactions=train_matrix, 
#     epochs=15,
#     num_threads=20
# );
# with open("lfm_model.pkl", 'wb') as f:
#     pickle.dump(lfm_model, f)

with open("lfm_model.pkl", 'rb') as f:
    lfm_model = pickle.load(f)

## Рекомендации для аватаров

In [10]:
n_recommendations = 10

In [11]:
id_item_mapping = {v: k for k, v in lfm_dataset._item_id_mapping.items()}

In [12]:
def get_n_recommendations_for_user(
    user_id: str,
    model: LightFM,
    train_matrix: coo_matrix,
    user_to_id: tp.Dict[str, int],
    id_to_item: tp.Dict[int, str],
    n_recommendations: int
) -> pd.DataFrame:
    user_inner_id = user_to_id[user_id]
    scores = model.predict(
        user_ids=user_inner_id,
        item_ids=np.arange(train_matrix.shape[1]),
        num_threads=20
    )
    user_watched_items = train_matrix.col[train_matrix.row == user_inner_id]
    scores[user_watched_items] = -np.inf

    recommended_item_inner_ids = np.argpartition(scores, -np.arange(n_recommendations))[
        -n_recommendations:
    ][::-1]
    recommended_item_ids = [id_to_item[x] for x in recommended_item_inner_ids]
    return recommended_item_ids


In [16]:
user_id = "avatar_action"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(items[["item_id", "title", "genres"]])

,user_id,item_id,title,genres
0,avatar_action,9728,Гнев человеческий,"боевики, триллеры"
1,avatar_action,13865,Девятаев,"драмы, военные, приключения"
2,avatar_action,12173,Мстители: Финал,"боевики, драмы, фантастика"
3,avatar_action,10440,Хрустальный,"триллеры, детективы"
4,avatar_action,14317,Веном,"популярное, фантастика, триллеры, боевики, ужасы"
5,avatar_action,7626,Мстители: Война бесконечности,"боевики, фантастика, приключения"
6,avatar_action,10942,Мстители,"боевики, фантастика, фэнтези, приключения"
7,avatar_action,3734,Прабабушка легкого поведения,комедии
8,avatar_action,5693,Алита: Боевой ангел,"боевики, фантастика, триллеры, приключения"
9,avatar_action,14488,Мастер меча,"боевики, историческое"


In [17]:
user_id = "avatar_comedy"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(items[["item_id", "title", "genres"]])

,user_id,item_id,title,genres
0,avatar_comedy,9728,Гнев человеческий,"боевики, триллеры"
1,avatar_comedy,13865,Девятаев,"драмы, военные, приключения"
2,avatar_comedy,10440,Хрустальный,"триллеры, детективы"
3,avatar_comedy,15297,Клиника счастья,"драмы, мелодрамы"
4,avatar_comedy,3734,Прабабушка легкого поведения,комедии
5,avatar_comedy,2657,Подслушано,"драмы, триллеры"
6,avatar_comedy,7571,100% волк,"мультфильм, приключения, семейное, фэнтези, ко..."
7,avatar_comedy,4151,Секреты семейной жизни,комедии
8,avatar_comedy,12192,Фемида видит,"драмы, детективы, комедии"
9,avatar_comedy,4880,Афера,комедии


In [18]:
# самые просматриваемые в обучающей выборке
train["title"].value_counts().head(10)

Клиника счастья                 208652
Хрустальный                     208480
Гнев человеческий               159456
Девятаев                        136208
Прабабушка легкого поведения     85098
Подслушано                       66534
Секреты семейной жизни           63962
Фемида видит                     58817
Коса                             53462
Афера                            35817
Name: title, dtype: int64

Мы получили, что рекомендации для аватаров имеют сильное пересечение, обусловленное перекосом к рекомендациям популярного контента.

##  Попробуем побороться с перекосом к популярным

Алгоритм обучает для каждого пользователя $u$ и товара $i$ соответственно смещения $b_u$, $b_i$ и эмбеддинги $p_u$, $q_i$. Для формирования рекомендаций для пользователя выбираются товары, имеющие наибольшие значения скоров, определяющихся по формуле:

$$score_{ui} = b_u + b_i + p_u \cdot q_i = b_u + b_i + \cos ( p_u, q_i ) \cdot || p_u || \cdot || q_i || .$$

Часто перекос к популярным выражается в больших значениях смещений или норм эмбеддингов у популярных товаров. В таких случаях может помочь переход от ранжирования по значениям скалярных произведений к ранжированию по косинусам угла между эмбедднгами пользователей и товаров.

Для перехода к косинусам согласно формуле выше достаточно заменить $b_u$ и $b_i$ нулями и привести нормы $p_u$ и $q_i$ к единицам.

In [19]:
lfm_model_cos = deepcopy(lfm_model)

lfm_model_cos.item_biases = np.zeros_like(lfm_model_cos.item_biases)
lfm_model_cos.user_biases = np.zeros_like(lfm_model_cos.user_biases)

lfm_model_cos.item_embeddings = normalize(lfm_model_cos.item_embeddings)
lfm_model_cos.user_embeddings = normalize(lfm_model_cos.user_embeddings)

In [21]:
user_id = "avatar_action"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model_cos,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(items[["item_id", "title", "genres"]])

,user_id,item_id,title,genres
0,avatar_action,3404,РЭД,"боевики, триллеры, криминал, комедии"
1,avatar_action,1811,Скала,"боевики, триллеры"
2,avatar_action,3594,Риддик,"боевики, фантастика, триллеры, приключения"
3,avatar_action,5482,Неудержимый,"боевики, триллеры, криминал"
4,avatar_action,10058,Элизиум: Рай не на Земле,"боевики, драмы, фантастика"
5,avatar_action,8430,Армагеддон,"боевики, триллеры"
6,avatar_action,1247,Терминатор: Да придёт спаситель,"боевики, фантастика, приключения"
7,avatar_action,6384,Адреналин: высокое напряжение,"боевики, фэнтези, криминал, комедии"
8,avatar_action,3345,Рэмбо 4,"боевики, триллеры"
9,avatar_action,4417,Хищник 2,"боевики, ужасы, фантастика"


In [22]:
user_id = "avatar_comedy"

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model_cos,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(items[["item_id", "title", "genres"]])

,user_id,item_id,title,genres
0,avatar_comedy,7330,Дядюшка Бак,"драмы, комедии"
1,avatar_comedy,8000,Дом с паранормальными явлениями 2,комедии
2,avatar_comedy,8357,Стой! Или моя мама будет стрелять,"боевики, криминал, комедии"
3,avatar_comedy,12118,Чокнутый профессор,"фантастика, мелодрамы, комедии"
4,avatar_comedy,6492,Детсадовский полицейский 2,"боевики, комедии"
5,avatar_comedy,6779,Детсадовский полицейский,комедии
6,avatar_comedy,4359,Американский пирог: Книга любви,комедии
7,avatar_comedy,2483,Очень страшное кино 5,комедии
8,avatar_comedy,7558,Дадли Справедливый,"мелодрамы, семейное, комедии"
9,avatar_comedy,9163,Большой толстый лгун,"семейное, приключения, комедии"


## Расчет рекомендаций

In [23]:
models_dict = {"lfm": lfm_model, "lfm_cos": lfm_model_cos}

In [24]:
%%time

recommendations_dict = {}
for model_name, model in models_dict.items():
    recommendations = pd.DataFrame({"user_id": test["user_id"].unique()})
    recommendations["item_id"] = recommendations["user_id"].apply(
        get_n_recommendations_for_user,
        args=(
            model,
            train_matrix,
            lfm_dataset._user_id_mapping,
            id_item_mapping,
            n_recommendations
        ),
    )
    recommendations = recommendations.explode("item_id")
    recommendations["rank"] = recommendations.groupby(["user_id"]).cumcount() + 1
    recommendations_dict[model_name] = recommendations
    

CPU times: user 35min 40s, sys: 7.03 s, total: 35min 47s
Wall time: 1min 48s


# Оценка beyond accuracy метрик
## Intra-List Diversity

In [25]:
# one-hot жанры
item_genres_one_hot = items[["item_id", "genres"]].copy()
item_genres_one_hot["genres"] = item_genres_one_hot["genres"].str.split(", ")
item_genres_one_hot = item_genres_one_hot.explode("genres")
item_genres_one_hot["genres"] = item_genres_one_hot["genres"].str.replace(" ", "_")
item_genres_one_hot["genres"] = item_genres_one_hot["genres"].map(lambda x: translit(x, "ru", reversed=True))
item_genres_one_hot["value"] = 1
item_genres_one_hot = item_genres_one_hot.pivot(
    index="item_id", 
    columns="genres", 
    values="value"
).fillna(0).astype(int)

item_genres_one_hot.head()

genres,18+,animatsija,anime,arthaus,biografija,bloger,boeviki,detektivy,detskie,detskie_pesni,...,trillery,uvlechenija,uzhasy,vestern,voennye,vokrug_sveta,vospitanie_detej,zapadnye_mul'tfil'my,zarubezhnye,zhivaja_priroda
item_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
def get_hamming_distances(pairs: pd.Series, features: pd.DataFrame) -> np.ndarray:
    items_0 = pairs.map(lambda pair: pair[1]).values
    items_1 = pairs.map(lambda pair: pair[0]).values

    features_0 = features.reindex(items_0).values
    features_1 = features.reindex(items_1).values
    return np.sum(features_0 != features_1, axis=1)


def calculate_intra_list_diversity_per_user(recommendations: pd.DataFrame, features: pd.DataFrame) -> pd.Series:
    recommended_item_pairs = recommendations.groupby("user_id")["item_id"].apply(
        lambda x: list(combinations(x, 2))
    ).reset_index().explode("item_id").rename(columns={"item_id": "item_pair"})
    recommended_item_pairs["dist"] = get_hamming_distances(recommended_item_pairs["item_pair"], features)
    return recommended_item_pairs[["user_id", "dist"]].groupby("user_id").agg("mean")


In [27]:
for model_name, recommendations in recommendations_dict.items():
    ild_per_user = calculate_intra_list_diversity_per_user(recommendations, item_genres_one_hot)
    print(f"model: {model_name}, mean ild: {round(float(ild_per_user.mean()), 2)}\n")
    

model: lfm, mean ild: 3.24

model: lfm_cos, mean ild: 2.96



## Mean Inverse User Frequency

In [29]:
def calculate_mean_inv_user_frequency_per_user(recommendations: pd.DataFrame, train: pd.DataFrame) -> pd.Series:
    n_users = train["user_id"].nunique()
    n_users_per_item = train.groupby("item_id")["user_id"].nunique()
    
    recommendations_ = recommendations[["user_id", "item_id"]].copy()
    recommendations_["n_users_per_item"] = recommendations_["item_id"].map(n_users_per_item)
    recommendations_["inv_user_freq"] = -np.log2(recommendations_["n_users_per_item"] / n_users)
    return recommendations_[["user_id", "inv_user_freq"]].groupby("user_id").agg("mean")

In [30]:
for model_name, recommendations in recommendations_dict.items():
    miuf_per_user = calculate_mean_inv_user_frequency_per_user(recommendations, train)
    print(f"model: {model_name}, mean miuf: {round(float(miuf_per_user.mean()), 2)}\n")

model: lfm, mean miuf: 4.34

model: lfm_cos, mean miuf: 10.21



## Serendipity

In [32]:
def get_value_popularity_ranks(values: pd.Series) -> pd.Series:
    value_counts = values.value_counts()
    counts_unique = value_counts.unique()
    count_rank_mapping = pd.Series(index=counts_unique, data=np.arange(len(counts_unique)) + 1)
    return value_counts.map(count_rank_mapping)


def calculate_serendipity_per_user(
    recommendations: pd.DataFrame, 
    train: pd.DataFrame,
    test: pd.DataFrame,
) -> pd.Series:
    relevant_recommendations = pd.merge(test[["user_id", "item_id"]], recommendations)
    
    n_items = train["item_id"].nunique()
    item_popularity_ranks = get_value_popularity_ranks(train["item_id"])
    relevant_recommendations["rank_pop"] = relevant_recommendations["item_id"].map(item_popularity_ranks)
    
    relevant_recommendations["proba_user"] = (n_items + 1 - relevant_recommendations["rank"]) / n_items
    relevant_recommendations["proba_any_user"] = (n_items + 1 - relevant_recommendations["rank_pop"]) / n_items
    
    relevant_recommendations["serendipity"] = np.maximum(
        relevant_recommendations["proba_user"] - relevant_recommendations["proba_any_user"],
        0.0
    )
    serendipity_per_user = relevant_recommendations[["user_id", "serendipity"]].groupby("user_id").agg("mean")
    
    result = pd.Series(index=recommendations["user_id"].unique(), data=0.0)
    result.loc[serendipity_per_user.index] = serendipity_per_user.squeeze()
    return result


In [33]:
for model_name, recommendations in recommendations_dict.items():
    serendipity_per_user = calculate_serendipity_per_user(recommendations, train, test)
    print(f"model: {model_name}, mean serendipity: {round(float(serendipity_per_user.mean()), 5)}\n")

model: lfm, mean serendipity: 0.00028

model: lfm_cos, mean serendipity: 0.00109



In [ ]:
# показать рекомендацию с макс serendipity

In [ ]:
# поменять формулы на слайде: заменить R на I, изменить формулу ранга и указать что он от 0